In [1]:
import sys
sys.path.append('..')
from model import Model
import torch
from PIL import Image
import numpy as np

In [2]:
model = Model()
model.load_state_dict(torch.load("../model_0399.pth"))

<All keys matched successfully>

In [3]:
# Load an example image
image_path = "../data/CityScapes/leftImg8bit/train/aachen/aachen_000001_000019_leftImg8bit.png"
mask_path = "../data/CityScapes/gtFine/train/aachen/aachen_000001_000019_gtFine_labelIds.png"
image = Image.open(image_path)
mask = Image.open(mask_path)

# Show the image
image.show()
mask.show()

In [4]:
from process_data import joint_preprocess

# Preprocess the image
image_t, mask_t = joint_preprocess(image, mask)

# Check shapes
print(image_t.shape, mask_t.shape)
print(image_t.dtype, mask_t.dtype)
print(image_t.min(), image_t.max(), mask_t.min(), mask_t.max())

# Show the preprocessed image
image_t = image_t.permute(1, 2, 0)
image_t = image_t.numpy()
image_t = (image_t * 255).astype(np.uint8)
image_t = Image.fromarray(image_t)
image_t.show()

# Show the preprocessed mask
print(mask_t)


torch.Size([3, 256, 256]) torch.Size([1, 256, 256])
torch.float32 torch.uint8
tensor(0.) tensor(1.) tensor(0, dtype=torch.uint8) tensor(26, dtype=torch.uint8)
Image([[[23, 23, 23,  ...,  0,  0,  0],
        [23, 23, 23,  ...,  0,  0,  0],
        [23, 23, 23,  ...,  0,  0,  0],
        ...,
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0],
        [ 0,  0,  0,  ...,  0,  0,  0]]], dtype=torch.uint8, )


In [5]:


# Make a prediction
with torch.no_grad():
    prediction = model(image_t)
    

TypeError: conv2d() received an invalid combination of arguments - got (Image, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!Image!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!Image!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, int)


In [ ]:
print(prediction.shape)
print(prediction)

In [ ]:
import torch.nn.functional as F
prediction_smax = F.softmax(prediction, dim=1)
print(prediction_smax.shape)
print(prediction_smax)

In [ ]:
prediction_argmax = torch.argmax(prediction_smax, dim=1)

print(prediction_argmax.shape)
print(prediction_argmax)
print(np.unique(prediction_argmax))

In [ ]:
# Postprocess the prediction
prediction = postprocess(prediction)

# Check size of prediction numpy array
print(prediction.shape)
print(np.unique(prediction))

In [ ]:
# Turn the prediction into an image
prediction_s = prediction.squeeze(0)
prediction_image = Image.fromarray(prediction_s)
prediction_image.show()